In [105]:
import glob
import os
import pathlib
import frontmatter
import re
import collections

import yaml


#------------------BEGIN HACKY --------------
_mapping_tag = yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG

def dict_representer(dumper, data):
    return dumper.represent_dict(data.items())

def dict_constructor(loader, node):
    loader.flatten_mapping(node)
    return collections.OrderedDict(loader.construct_pairs(node))

def null_representer(dumper, value):
    return dumper.represent_scalar(u'tag:yaml.org,2002:null', '')

try:
    yaml.CSafeDumper.add_representer(collections.OrderedDict, dict_representer)
    yaml.CSafeDumper.add_representer(type(None), null_representer)
except (AttributeError, ImportError):
    pass



yaml.SafeDumper.add_representer(collections.OrderedDict, dict_representer)
yaml.SafeDumper.add_representer(type(None), null_representer)
yaml.SafeLoader.add_constructor(_mapping_tag, dict_constructor)

#------------------END HACKY --------------


content = glob.glob('content/**/*.md')





In [ ]:

for sourcefile in content:
    basename = os.path.basename(sourcefile)
    if basename == '_index.md':
        continue
    post = frontmatter.load(sourcefile)
    year = post.metadata['date'].year
    targetfolder = re.findall(r'^(\w+[/\\]\w+)',os.path.dirname(sourcefile))[0] 
    targetfolder = os.path.join(targetfolder,f'{year:.0f}')
    targetfile = os.path.join(targetfolder,basename)
    #print(targetfile,sourcefile)
    post.metadata = collections.OrderedDict(post.metadata)
    post.metadata.move_to_end('title', last=False)
    #then reverse
    
    os.makedirs(targetfolder, exist_ok=True)

    banner = post.metadata['banner']
    images = re.findall(r'!\[[^\]]*\]\((.*?)\s*("(?:.*[^"])")?\s*\)',post.content) #https://stackoverflow.com/questions/44227270/regex-to-parse-image-link-in-markdown
    images.append((banner,""))

    outputpost = frontmatter.dumps(post)
    for source_image_url,optionaltxt in images:
        target_image = os.path.join(targetfolder, os.path.basename(source_image_url))
        target_image_url = target_image.replace('\\','/').replace('content','')
        if source_image_url[0]=='/':
            source_image = os.path.join('static',source_image_url[1:])
        else:
            print('NOT MOVING',source_image_url)
            continue
        #print('FILE:', source_image, '->', target_image)
        #print('URL:', source_image_url, '->', target_image_url)

        if not os.path.exists(source_image):
            print('path does not exist', source_image)
            continue

        outputpost = outputpost.replace(source_image_url,target_image_url)
        
        os.rename(source_image,target_image)

    os.remove(sourcefile)
    with open(targetfile, "w") as text_file:
        text_file.write(outputpost)



    
    #if len(images)>4:
    #    break

    #orig_bannerimg = 
    #print()
    #break

AttributeError: module 'ntpath' has no attribute 'exist'

--- title: Return period of Boulder 2013 extreme rain date: 2013-11-20
01:30:00+00:00 author: aslak banner: XXXXXXXXXXXXXXXXXX aliases: -
/2013/11/20/return-period-of-boulder-2013-extreme-rain/ -
/Home/Miscellaneous-Debris/returnperiodofboulder2013extremerain ---
!![](XXXXXXXXXXXXXXXXXX)Several media stories on the Colorado floods
talk of it as a 1000-year flood. However, the original source of this
claim was talking about a 1000-year precip event (not the same as a
flood-event). Pielke Jr discusses this on [his
blog](http://rogerpielkejr.blogspot.dk/2013/09/how-fantasy-becomes-
fact.html?m=1). <!--more--> The source for the 1000-year return level
estimates appears to be this [NOAA](http://hdsc.nws.noaa.gov/hdsc/pfds
/pfds_map_cont.html?bkmrk=co) page. We can argue about how solid these
very rare return levels are as they obviously must be based on an
extrapolation beyond the largest recorded event. Nevertheless out of
curiosity I looked up what return period NOAA estimated for the
obs

<function OrderedDict.__reversed__>

TypeError: YAMLHandler.export() missing 1 required positional argument: 'metadata'

In [13]:
post = frontmatter.load(sourcefile)

In [18]:
post.metadata['banner']

'/2010/02/sia.jpg'

In [ ]:
post.